### 로드

In [1]:
# %pip install chromadb sentence_transformers gdown dotenv openai

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
import chromadb
from sentence_transformers import SentenceTransformer
import torch
import os
from chromadb.utils import embedding_functions


# ---------------------------------------------------------
# 1. 설정 (GPU 확인 등)
# ---------------------------------------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# ---------------------------------------------------------
# 2. 모델 로드 (저장할 때 썼던 그 모델!)
# ---------------------------------------------------------
print("모델 로드 중...")
doc_model = SentenceTransformer("dragonkue/BGE-m3-ko").to(device)

ipc_model = embedding_functions.OpenAIEmbeddingFunction(
    api_key=os.getenv("OPENAI_API_KEY"),
    model_name="text-embedding-3-small",
)

# ---------------------------------------------------------
# 3. ChromaDB 연결 (다운로드 받은 폴더 경로 지정)
# ---------------------------------------------------------
doc_client = chromadb.PersistentClient(path="./doc_db")
doc_collection = doc_client.get_collection(name="patent_claims")

ipc_client = chromadb.PersistentClient(path="./ipc_db")
ipc_collection = ipc_client.get_collection(name="ipc_clean")

print(f"✅ doc 데이터베이스 로드 완료! 총 데이터 수: {doc_collection.count()}개")
print(f"✅ ipc 데이터베이스 로드 완료! 총 데이터 수: {ipc_collection.count()}개")

c:\Users\Playdata\.conda\envs\3rdpro_env\Lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cpu
모델 로드 중...
✅ doc 데이터베이스 로드 완료! 총 데이터 수: 589049개
✅ ipc 데이터베이스 로드 완료! 총 데이터 수: 69337개


### 검색

In [4]:
# ---------------------------------------------------------
# 4. Doc DB 확인용 함수 (view)
# ---------------------------------------------------------
def search_doc(query_text, top_k=10):
    # 1) 검색어를 벡터로 변환 (임베딩)
    query_vector = doc_model.encode(query_text).tolist()

    # 2) 벡터로 DB 검색
    results = doc_collection.query(query_embeddings=[query_vector], n_results=top_k)

    # 3) 결과 출력
    print(f"\n🔍 검색어: '{query_text}'")

    for i in range(top_k):
        doc = results["documents"][0][i]
        meta = results["metadatas"][0][i]
        distance = results["distances"][0][i]

        print(f"--- [Rank {i+1}] (거리: {distance:.4f}) ---")
        print(f"특허명: {meta.get('title', '제목없음')}")
        print(f"특허ID: {meta.get('patent_id')} (Claim {meta.get('claim_no')})")
        print(f"내용: {doc}")


search_doc("환자의 신체 부위 내에서 내시경을 위치 파악하기 위한 방법")


🔍 검색어: '환자의 신체 부위 내에서 내시경을 위치 파악하기 위한 방법'
--- [Rank 1] (거리: 0.2531) ---
특허명: 내시경 위치 파악 시스템 및 방법
특허ID: 1020257025260 (Claim 1)
내용: 환자의 신체 부위 내에서 내시경을 위치 파악하기 위한 방법으로서,(a) 일련의 EM(electromagnetic) 데이터를 획득하는 단계;(b) 상기 일련의 EM 데이터에 적어도 부분적으로 기초하여 EM 데이터 기반 경로를 생성하는 단계;(c) 상기 일련의 EM 데이터의 데이터 포인트에 적어도 부분적으로 기초하여, 상기 환자의 신체 부위의 생물학적 모델에 기초한 형상을 갖는 하나 이상의 경로 가설을 식별하는 단계;(d) 최적화 알고리즘을 이용하여 상기 하나 이상의 경로 가설을 상기 EM 데이터 기반 경로에 매핑함으로써 하나 이상의 변형된 경로를 생성하는 단계; 및(e) 상기 하나 이상의 경로 가설 각각과 관련된 확률에 적어도 부분적으로 기초하여 상기 하나 이상의 변형된 경로로부터 변형된 경로를 선택하고, 그 선택된 변형된 경로에 기초하여 상기 내시경의 팁의 위치를 결정하는 단계를 포함하는, 방법.
--- [Rank 2] (거리: 0.3384) ---
특허명: 내시경 위치 파악 시스템 및 방법
특허ID: 1020257025260 (Claim 17)
내용: 환자의 신체 부위 내에서 내시경을 위치 파악하기 위한 시스템으로서,컴퓨터 실행 가능 명령어를 저장하는 메모리; 및상기 내시경과 통신하고 상기 컴퓨터 실행 가능 명령어를 실행하여, (a) 일련의 EM(electromagnetic) 데이터를 획득하고; (b) 상기 일련의 EM 데이터에 적어도 부분적으로 기초하여 EM 데이터 기반 경로를 생성하며; (c) 상기 일련의 EM 데이터의 데이터 포인트에 적어도 부분적으로 기초하여, 상기 환자의 신체 부위의 생물학적 모델에 기초한 형상을 갖는 하나 이상의 경로 가설을 식별하고;  (d) 최적화 알고리즘을 이용하여 상기 하나 이상의 경로 가설을 

In [5]:
# ---------------------------------------------------------
# 5. Doc DB 확인용 함수 (view)
# ---------------------------------------------------------
def search_ipc(query_text, top_k=10):
    # 1) 검색어를 벡터로 변환 (임베딩)
    query_vector = ipc_model(query_text)[0].tolist()

    # 2) 벡터로 DB 검색
    results = ipc_collection.query(query_embeddings=[query_vector], n_results=top_k)

    # 3) 결과 출력
    print(f"\n🔍 검색어: '{query_text}'")

    for i in range(top_k):
        ipc_explain = results["documents"][0][i]
        meta = results["metadatas"][0][i]
        ids = results["ids"][0][i]
        distance = results["distances"][0][i]

        print(f"--- [Rank {i+1}] (거리: {distance:.4f}) ---")
        print(f"코드: {ids} (Path {meta.get('path')})")
        print(f"분류: {meta.get('kind')}")
        print(f"내용: {ipc_explain}")


search_ipc(
    "COMPUTER VISION-BASED ALZHEIMER'S DIAGNOSIS SYSTEM AND METHOD USING SUBJECT'S PUPIL MOVEMENT AND MICROSACCADE"
)


🔍 검색어: 'COMPUTER VISION-BASED ALZHEIMER'S DIAGNOSIS SYSTEM AND METHOD USING SUBJECT'S PUPIL MOVEMENT AND MICROSACCADE'
--- [Rank 1] (거리: 0.9906) ---
코드: G06V10/10 (Path G > G06 > G06V > G06V10/00 > G06V10/10)
분류: 1
내용: PHYSICS COMPUTING CALCULATING OR COUNTING IMAGE OR VIDEO RECOGNITION OR UNDERSTANDING Arrangements for image or video recognition or understanding Image acquisition
--- [Rank 2] (거리: 0.9942) ---
코드: G06T7/23 (Path G > G06 > G06T > G06T7/00 > G06T7/20 > G06T7/22 > G06T7/23)
분류: 3
내용: PHYSICS COMPUTING CALCULATING OR COUNTING IMAGE DATA PROCESSING OR GENERATION, IN GENERAL Image analysis Analysis of motion using block-matching using full search
--- [Rank 3] (거리: 1.0011) ---
코드: G06V (Path G > G06 > G06V)
분류: u
내용: PHYSICS COMPUTING CALCULATING OR COUNTING IMAGE OR VIDEO RECOGNITION OR UNDERSTANDING
--- [Rank 4] (거리: 1.0139) ---
코드: G06V30/14 (Path G > G06 > G06V > G06V30/00 > G06V30/10 > G06V30/14)
분류: 2
내용: PHYSICS COMPUTING CALCULATING OR COUNTING IMAGE OR VIDEO RECOGNI

### 임계값 설정

In [6]:
import numpy as np


def get_universal_threshold(collection, k=100):
    """
    IPC 전체 섹션(A~H)을 아우르는 테스트 쿼리를 실행하여,
    모든 분야에 적용 가능한 '단일 통합 임계값(Ratio 95%)'을 산출합니다.
    """

    # 1. 분야별 대표 쿼리셋 (IPC Section A ~ H 전체 포함)
    # 이 쿼리들이 기계, 화학, 바이오, 전기 등 서로 다른 벡터 분포를 모두 건드려봅니다.
    universal_queries = [
        # [A] 생활/의료 (보통 중간 길이 텍스트)
        "artificial intelligence alzheimer diagnosis system",
        "agricultural harvesting machine mechanism",
        # [B] 운수/처리 (구조적, 기계적)
        "autonomous vehicle steering control system",
        "plastic waste recycling process separation",
        # [C] 화학/제철 (텍스트가 길고 복잡함 -> Gap이 클 수 있음)
        "lithium ion battery electrolyte composition",
        "organic light emitting diode structure OLED",
        # [D] 섬유/제지
        "waterproof breathable textile fabric manufacturing",
        # [E] 고정구조물 (건축)
        "earthquake resistant building damper device",
        # [F] 기계공학 (짧고 명확함 -> Gap이 작을 수 있음)
        "hydraulic piston pump valve assembly",
        "wind turbine blade aerodynamic design",
        # [G] 물리학/컴퓨터 (추상적 개념 포함)
        "image recognition neural network deep learning",
        "quantum computing error correction algorithm",
        # [H] 전기 (매우 구체적)
        "wireless power transmission coil alignment",
        "semiconductor memory device 3D nand flash",
    ]

    print(f"🚀 통합 분석 시작: 총 {len(universal_queries)}개 분야 테스트 중...")

    all_ratios = []

    # 2. 시뮬레이션 실행
    for query in universal_queries:
        query_vector = ipc_model(query)[0].tolist()
        results = collection.query(
            query_embeddings=[query_vector],
            n_results=k,
            include=["metadatas", "distances"],
        )

        if not results["ids"] or not results["ids"][0]:
            continue

        ids = results["ids"][0]
        dists = results["distances"][0]
        metas = results["metadatas"][0]

        # 데이터 매핑
        data_map = {
            id_: {"dist": dist, "meta": meta}
            for id_, dist, meta in zip(ids, dists, metas)
        }

        # 부모-자식 쌍 찾기
        for child_code, info in data_map.items():
            path_str = info["meta"].get("path", "")  # 예: "A > A61 > A61B..."
            if not path_str:
                continue

            # Path 파싱 (조상 코드 추출)
            ancestors = [
                x.strip() for x in path_str.replace(">", " > ").split(">") if x.strip()
            ]

            # 자기 자신 제외
            if child_code in ancestors:
                ancestors.remove(child_code)

            # 검색 결과에 조상이 있는지 확인
            for parent_code in ancestors:
                if parent_code in data_map:
                    parent_dist = data_map[parent_code]["dist"]
                    child_dist = info["dist"]

                    # 비율(Ratio) 계산
                    # 절대값(Gap)을 부모 거리로 나누어 %로 환산
                    gap = abs(child_dist - parent_dist)
                    if parent_dist > 0:
                        ratio = (gap / parent_dist) * 100
                        all_ratios.append(ratio)

    if not all_ratios:
        print("❌ 데이터 부족: 부모-자식 쌍이 발견되지 않았습니다. (k값을 늘려보세요)")
        return None

    # 3. 95번째 백분위수 계산 (이것이 바로 '통합 값'입니다)
    final_threshold = np.percentile(all_ratios, 95)

    print("\n" + "=" * 60)
    print(f"🏆 [분석 완료] 총 {len(all_ratios)}개의 부모-자식 관계를 분석했습니다.")
    print("-" * 60)
    print(f"👉 모든 분야 통일 추천 임계값 (Ratio 95%): {final_threshold:.2f}%")
    print("-" * 60)
    print("✅ 사용법:")
    print(f"   코드 상단에 다음 상수를 선언하세요.")
    print(f"   MERGE_THRESHOLD_RATIO = {final_threshold:.2f}")
    print("=" * 60)

    return final_threshold


# --- 실행 ---
optimal_value = get_universal_threshold(ipc_collection, k=50)

🚀 통합 분석 시작: 총 14개 분야 테스트 중...

🏆 [분석 완료] 총 945개의 부모-자식 관계를 분석했습니다.
------------------------------------------------------------
👉 모든 분야 통일 추천 임계값 (Ratio 95%): 6.20%
------------------------------------------------------------
✅ 사용법:
   코드 상단에 다음 상수를 선언하세요.
   MERGE_THRESHOLD_RATIO = 6.20


In [7]:
MERGE_THRESHOLD_RATIO = 6.64

### 오염처리 로직 적용 검색

In [8]:
import os

# chroma_db와 embedding_functions는 이미 선언되어 있다고 가정합니다.

# ✅ 상수 설정
MERGE_THRESHOLD_RATIO = 6.64  # 통합 임계값 (비율 %)
TOP_K = 50  # 검색 개수 (Oversampling)
MAX_DISTANCE_THRESHOLD = (
    1.4  # ⛔ 거리 컷오프 (이 값보다 멀면 노이즈로 간주하고 즉시 폐기)
)


def get_ipc_codes_by_query(query_text, top_k=5):
    """
    단일 쿼리 텍스트를 받아 ChromaDB에서 검색 후,
    1. 거리(Distance) 기반 노이즈 필터링
    2. 계층적 중복 제거(병합)
    를 수행하여 구조화된 리스트를 반환합니다.
    """

    # ---------------------------------------------------------
    # 1. 임베딩 생성
    # ---------------------------------------------------------
    try:
        # ipc_model이 리스트 입력을 기대하므로 리스트로 감싸서 전달
        query_vector = ipc_model([query_text])[0]

        if hasattr(query_vector, "tolist"):
            query_vector = query_vector.tolist()

    except Exception as e:
        print(f"❌ 임베딩 생성 중 오류 발생: {e}")
        return []

    # ---------------------------------------------------------
    # 2. DB 검색 (Main Group 'm' 이하만 필터링)
    # ---------------------------------------------------------
    try:
        results = ipc_collection.query(
            query_embeddings=[query_vector],
            n_results=TOP_K,
            # 'kind'가 m(Main Group), 1~5(Subgroup)인 것만 검색
            where={"kind": {"$in": ["m", "1", "2", "3", "4", "5"]}},
            include=["metadatas", "distances"],
        )
    except Exception as e:
        print(f"❌ ChromaDB 검색 중 오류 발생: {e}")
        return []

    if not results["ids"] or not results["ids"][0]:
        return []

    raw_ids = results["ids"][0]
    raw_distances = results["distances"][0]
    raw_metadatas = results["metadatas"][0]

    # ---------------------------------------------------------
    # 3. [New] 노이즈 필터링 및 데이터 구조화
    # ---------------------------------------------------------
    # 거리가 너무 먼(MAX_DISTANCE_THRESHOLD 초과) 항목은 여기서 미리 걸러냅니다.

    code_map = {}
    valid_ids = []  # 순서 유지를 위한 리스트

    for code, dist, meta in zip(raw_ids, raw_distances, raw_metadatas):
        # ⛔ 거리 컷오프 적용 (예: 알루미늄 1.58 -> 제거됨)
        if dist > MAX_DISTANCE_THRESHOLD:
            continue

        valid_ids.append(code)
        code_map[code] = {"dist": dist, "meta": meta, "sub": [], "is_absorbed": False}

    # 유효한 결과가 하나도 없으면 빈 리스트 반환
    if not valid_ids:
        return []

    # ---------------------------------------------------------
    # 4. 지능형 그룹핑 로직 (Ratio <= 6.64%)
    # ---------------------------------------------------------
    for current_code in valid_ids:
        child_item = code_map[current_code]

        # 이미 흡수된 항목은 패스
        if child_item["is_absorbed"]:
            continue

        path_str = child_item["meta"].get("path", "")
        if not path_str:
            continue

        # 조상 코드 파싱
        ancestors = [
            x.strip() for x in path_str.replace(">", " > ").split(">") if x.strip()
        ]
        if current_code in ancestors:
            ancestors.remove(current_code)

        for parent_code in ancestors:
            # 조상 코드가 '유효한 검색 결과(valid_ids)' 내에 있을 때만 계산
            if parent_code in code_map:
                parent_item = code_map[parent_code]

                child_dist = child_item["dist"]
                parent_dist = parent_item["dist"]

                # 비율 계산
                if parent_dist > 0:
                    gap = abs(child_dist - parent_dist)
                    ratio = (gap / parent_dist) * 100

                    # 임계값 이하면 병합 (가족으로 인정)
                    if ratio <= MERGE_THRESHOLD_RATIO:
                        if parent_code not in child_item["sub"]:
                            child_item["sub"].append(parent_code)

                        # 조상은 흡수 처리 (메인 출력에서 제외)
                        code_map[parent_code]["is_absorbed"] = True

    # ---------------------------------------------------------
    # 5. 최종 결과 포맷팅
    # ---------------------------------------------------------
    final_output = []

    for code in valid_ids:
        item = code_map[code]

        if not item["is_absorbed"]:
            entry = {
                "main": code,
                "sub": item["sub"],
                "distance": item["dist"],
            }
            final_output.append(entry)

    return final_output[:top_k]

In [9]:
import numpy as np


def get_combined_ipc_codes(queries, total_top_k=5):
    """
    여러 개의 쿼리 문자열을 받아 통합된 IPC 코드 리스트를 반환합니다.

    전략: '품질 우선 라운드 로빈 (Quality-First Round Robin)'
    1. 각 쿼리별로 결과를 검색합니다.
    2. 결과 그룹별 '평균 거리(Average Distance)'를 계산하여 품질을 평가합니다.
    3. 품질이 좋은(거리가 가까운) 그룹부터 순서대로 1개씩 번갈아가며 뽑습니다.

    Args:
        queries (list): 검색할 쿼리 문자열 리스트 (예: ["컴퓨터 비전", "알츠하이머 진단"])
        total_top_k (int): 최종적으로 반환할 코드의 총 개수

    Returns:
        list: 통합 및 선별된 IPC 코드 딕셔너리 리스트
    """

    # 1. 쿼리별 결과 수집 및 그룹 품질 평가
    query_groups = []

    for query in queries:
        # 내부적으로는 넉넉하게 가져와야 선별이 가능하므로 total_top_k보다 많이 요청 (예: 3배수)
        # 여기서 get_ipc_codes_by_query는 이미 정의된 함수를 사용
        raw_results = get_ipc_codes_by_query(query, top_k=total_top_k * 3)

        if not raw_results:
            continue

        # 그룹의 품질 점수 계산 (상위 3개의 평균 거리)
        # 상위권 결과가 좋을수록 신뢰도 높은 그룹으로 판단
        top_n_check = min(len(raw_results), 3)
        avg_dist = (
            sum(item["distance"] for item in raw_results[:top_n_check]) / top_n_check
        )

        # 출처 태깅 (LLM이 어떤 쿼리에서 나온 건지 알 수 있게)
        for item in raw_results:
            item["source_query"] = query

        query_groups.append(
            {
                "query": query,
                "avg_dist": avg_dist,
                "queue": raw_results,  # 이미 거리순으로 정렬되어 있음
            }
        )

    # 검색 결과가 하나도 없는 경우
    if not query_groups:
        return []

    # 2. 그룹 정렬 (품질 좋은 순서: 거리 오름차순)
    # 예: [비전(0.9), AI(1.0), ..., 알루미늄(1.5)] 순서로 정렬됨
    query_groups.sort(key=lambda x: x["avg_dist"])

    final_list = []
    inserted_main_codes = set()  # 중복 방지용 집합

    # 3. 라운드 로빈으로 추출
    while len(final_list) < total_top_k:
        added_in_this_round = False

        # 품질 좋은 그룹부터 순회
        for group in query_groups:
            # 목표 개수 채웠으면 중단
            if len(final_list) >= total_top_k:
                break

            # 해당 그룹 큐에 남은 게 있다면
            if group["queue"]:
                # 큐에서 하나 꺼냄 (가장 상위 아이템)
                candidate = group["queue"].pop(0)

                # 중복 체크 (다른 쿼리에서 이미 뽑힌 코드가 아닐 경우만)
                if candidate["main"] not in inserted_main_codes:
                    final_list.append(candidate)
                    inserted_main_codes.add(candidate["main"])
                    added_in_this_round = True

        # 한 바퀴를 돌았는데도 추가된 게 없다면 (모든 큐가 비었음) 종료
        if not added_in_this_round:
            break

    return final_list

In [10]:
decomposited_queries = ["Computer Vision", "mental medical detect"]
a = get_combined_ipc_codes(decomposited_queries, 5)

In [11]:
temp_codes = {"mains": [], "subs": []}
for i in a:
    temp_codes["mains"].append(i.get("main"))
    if len(i.get("sub")) > 0:
        for ii in i.get("sub"):
            temp_codes["subs"].append(ii)
print(temp_codes)

{'mains': ['G06V10/76', 'A61B5/18', 'G06V20/58', 'A61B5/39', 'G06V10/60'], 'subs': ['A61B5/16', 'G06V20/56', 'A61B5/24', 'A61B5/31', 'A61B5/38']}


In [12]:
# ID(IPC코드) 리스트로 데이터 한방에 가져오기
def get_code(codes):
    results = ipc_collection.get(ids=codes)
    return results


display(get_code(["G", "G01"]))  # 해당 코드들의 설명 텍스트 출력

{'ids': ['G', 'G01'],
 'embeddings': None,
 'documents': ['PHYSICS', 'PHYSICS MEASURING TESTING'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [{'type': 'ipc', 'path': 'G', 'kind': 's'},
  {'parent': 'G', 'type': 'ipc', 'kind': 'c', 'path': 'G > G01'}]}

In [13]:
def get_ipc_data_from_code(codes):
    results = ipc_collection.get(ids=list(codes))
    returns = []
    for i in range(len(codes)):
        temp = {"ids": results.get("ids")[i], "document": results.get("documents")[i]}
        returns.append(temp)
    return returns

In [ ]:
def search_ipc_with_query(queries, top_k=5):
    search_output = get_combined_ipc_codes(queries, top_k)
    temp_codes = {"mains": [], "subs": []}
    for i in search_output:
        temp_codes["mains"].append(i.get("main"))
        if len(i.get("sub")) > 0:
            for ii in i.get("sub"):
                temp_codes["subs"].append(ii)
    returns = {"mains": get_ipc_data_from_code(set(temp_codes["mains"]))}
    if len(set(temp_codes["subs"])) > 0:
        returns["subs"] = (get_ipc_data_from_code(set(temp_codes["subs"])),)
    return returns

In [20]:
search_ipc_with_query(decomposited_queries, 7)

{'mains': [{'ids': 'A61B5/18',
   'document': 'HUMAN NECESSITIES MEDICAL OR VETERINARY SCIENCE HYGIENE DIAGNOSIS SURGERY IDENTIFICATION Measuring for diagnostic purposes Identification of persons Devices for psychotechnics Testing reaction times for vehicle drivers'},
  {'ids': 'A61B5/34',
   'document': 'HUMAN NECESSITIES MEDICAL OR VETERINARY SCIENCE HYGIENE DIAGNOSIS SURGERY IDENTIFICATION Measuring for diagnostic purposes Identification of persons Detecting, measuring or recording bioelectric or biomagnetic signals of the body or parts thereof for electroencephalography [EEG] for electroencephalography [EEG] specially adapted for cooperation with other devices Vectorcardiography [VCG]'},
  {'ids': 'A61B5/39',
   'document': 'HUMAN NECESSITIES MEDICAL OR VETERINARY SCIENCE HYGIENE DIAGNOSIS SURGERY IDENTIFICATION Measuring for diagnostic purposes Identification of persons Detecting, measuring or recording bioelectric or biomagnetic signals of the body or parts thereof for electroenc

In [ ]:
inputs = [
    "Organic Light Emitting Display",
    "Display Panel Opening Area",
    "Pixel Electrode Contact Structure",
    "Voltage Wiring Connection",
    "Pixel Definition Layer",
]
search_ipc_with_query(inputs, 10)

{'mains': [{'ids': 'F24F13/18',
   'document': 'MECHANICAL ENGINEERING LIGHTING HEATING WEAPONS BLASTING HEATING RANGES VENTILATING AIR-CONDITIONING AIR-HUMIDIFICATION VENTILATION USE OF AIR CURRENTS FOR SCREENING Details common to, or for air-conditioning, air-humidification, ventilation or use of air currents for screening Air-flow control members, e.g. louvres, grilles, flaps or guide plates specially adapted for insertion in flat panels, e.g. in door or window-pane'},
  {'ids': 'F24F120/12',
   'document': 'MECHANICAL ENGINEERING LIGHTING HEATING WEAPONS BLASTING HEATING RANGES VENTILATING AIR-CONDITIONING AIR-HUMIDIFICATION VENTILATION USE OF AIR CURRENTS FOR SCREENING Control inputs relating to users or occupants Occupancy Position of occupants'},
  {'ids': 'G06T11/80',
   'document': 'PHYSICS COMPUTING CALCULATING OR COUNTING IMAGE DATA PROCESSING OR GENERATION, IN GENERAL 2D [Two Dimensional] image generation Creating or modifying a manually drawn or painted image using a manua